# Lab Assignment 03 - Automatic Text Summarization Using Python

In this lab assignment, we will be using a variety of tools and techniques to automatically generate summaries of text documents.

By the time you have completed this lab, you will have achieved all of the following learning objectives:

## Learning Objectives

* Gain familiarity with the Natural Language Toolkit (NLTK).
* Learn how to split text into paragraphs and sentences.
* Understand the challenges associated with sentence boundary disambiguation.
* Learn how to generate feature vector representations of sentences (as opposed to documents).
* Be able to compute a centroid for the sentences in a document.
* Know how to compute Maximum Marginal Relevance (MMR) scores, and use those MMR scores to generate an extractive text summary.
* Learn how to split a sentence into n-grams.
* Know how to construct a language model based on n-grams.
* Understand how to calculate conditional probabilities for n-grams, and how those probabilities can be used to select the next word in a sequence.
* Be able to generate an abstractive text summary.
* Continue to develop skills working with and analyzing text in Python.

In [19]:
#import libraries
import nltk #the natural language toolkit
import numpy as np #used to generate random numbers
import pandas as pd #used to store data in a dataframe
import re #regular expressions; used to clean the text data
import string #used to determine if a character is a punctuation symbol
from nltk.probability import FreqDist #used to compute conditional frequency distributions
from nltk.tokenize import sent_tokenize, word_tokenize #used to split text into sentences, and to split sentences into words
from nltk.util import ngrams #used to generate n-grams for each sentences
from sklearn.feature_extraction.text import TfidfVectorizer #used to generate TF-IDF vectors and build the vocabulary
from sklearn.metrics.pairwise import cosine_similarity #used to compute cosine similarities

#install nltk's 'punkt' tools, which are needed to tokenize sentences and words
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

##Extractive Text Summarization
In ***extractive text summarization***, the summary consists of a sequence of words or sentences that have been selected and extracted from the original text. Extractive text summarization relies on *sentence vectors* (as opposed to document vectors) and similarity functions in order to create a summary of the original text.

The basic approach to extractive text summarization involves two steps:
1. Split the original text into smaller sections or passages of text.
2. For each passage of text, compress its sentences into a smaller number of sentences. This is accomplished by selecting the most representative sentence within the passage of text.


###Load Data
Run the code cell below to load the data for this part of the lab assignment. In this case, we'll be working with part of the Wikipedia article about the State of Washington.

In [20]:
#load raw text from a file
with open('Washington.txt', 'r') as f:
  text = f.read()

###Split Text Into Passages
The first step in the extractive text summarization process is to split the raw text into passages. The simplest way of splitting text into passages is to rely on the text's existing paragraphs. Since paragraphs are used to divide a written document into smaller pieces that address a single topic or concept, using paragraphs as the basis for defining passages of text for our extractive summarization task is very natural. 

Paragraphs begin on a new line within a document, so we can use Python's new line character `\n` (also called a *linefeed*) as a way of identifying the boundaries between paragraphs. 

**TASK 01**:
>Run the code cell below to split the raw text into paragraphs, then write a line of code that will display the total number of paragraphs in the raw text.

**QUESTION 01**:
>How many paragraphs are in the article about the State of Washington?

In [21]:
#split text into paragraphs
text = text.replace('\n\n', '\n') # replace any double linefeeds with a single linefeed
paragraphs = text.split('\n')

In [22]:
#display the number of paragraphs in the raw text
len(paragraphs)

5

###Split Passages Into Sentences
As noted above, extractive text summarization relies on *sentence vectors* (as opposed to document vectors). As such, we'll need to split each passage (paragraph) of text into sentences. **Don't be fooled into thinking that this is an easy task!** Accurately identifying the boundaries between sentences is a much subtler process than it may initially seem. At first, we may think that English has only three punctuation symbols that can end a sentence -- a period (full stop) `.`, an exclamation point `!`, and a question mark `?` -- and that we can therefore identify the end of a sentence just by looking for one of these symbols. Things are, however, not so simple. Consider the following sentences:
* I enjoy books written by George R. R. Martin.
* "What time will you arrive?", she asked.

If we used our simple rules, then the first sentence above would be treated as three separate sentences:
1. I enjoy books written by George R.
2. R.
3. Martin.

...and the second sentence above would be treated as two separate sentences:
1. "What time will you arrive?
2. ", she asked.

As you can see, this problem is not as easy as it seems! The task of identifying the boundaries between sentences is known as ***sentence boundary disambiguation***. If you'd like to learn more, then please read [this article](https://en.wikipedia.org/wiki/Sentence_boundary_disambiguation).

The good news is that researchers have created sophisticated tools that can accurately identify sentence boundaries. One of these tools is available in the **Natural Language Toolkit (NLTK)**, which we will use extensively in this lab assignment.

**TASK 02**:
>Run the code cell below to split each paragraph into sentences using NLTK's sentence tokenizer, then write some code that will calculate and display the average number of sentences per paragraph.

**QUESTION 02**:
>What is the average number of sentences per paragraph for the article about the State of Washington? Report your answer using three decimals of precision (e.g., 3.456).

In [23]:
#split each paragraph into sentences
for i in range(len(paragraphs)):
  paragraphs[i] = sent_tokenize(paragraphs[i])

#display the sentences in the first paragraph
paragraphs[0]

['Washington, officially the State of Washington, is a state in the Pacific Northwest region of the Western United States.',
 'Named for George Washington, the first U.S. president, the state was made out of the western part of the Washington Territory, which was ceded by the British Empire in 1846, in accordance with the Oregon Treaty in the settlement of the Oregon boundary dispute.',
 'The state, which is bordered on the west by the Pacific Ocean, Oregon to the south, Idaho to the east, and the Canadian province of British Columbia to the north, was admitted to the Union as the 42nd state in 1889.',
 'Olympia is the state capital.',
 "The state's largest city is Seattle.",
 "Washington is often referred to as Washington state to distinguish it from the nation's capital, Washington, D.C."]

In [30]:
#calculate and display the average number of sentences per paragraph
for i in range(len(paragraphs)):
  print(len(paragraphs[i]))
  #4.4

6
5
5
2
4


###Move Data to a DataFrame
Next, let's move all of our sentences into a pandas dataframe so that they'll be easier to work with.

Run the code cell below to add all of our sentences to a pandas dataframe. We'll also add a column named `sentence_id` that will record the order in which each sentence appeared in the original article, as well as a column named `paragraph_id` that will record the paragraph in which each sentence appeared in the original article.

In [31]:
#construct the rows of data
rows = []
sentence_id = 0
for paragraph_id in range(len(paragraphs)):
  for sentence in paragraphs[paragraph_id]:
    rows.append([sentence_id, paragraph_id, sentence])
    sentence_id += 1

#add the sentences to a dataframe
df = pd.DataFrame(rows, columns=['sentence_id', 'paragraph_id', 'raw_text'])
df.set_index('sentence_id', inplace=True) #use the sentence_id column as the dataframe's index

#display the first few rows
df.head(10)

,paragraph_id,raw_text
sentence_id,,
0,0,"Washington, officially the State of Washington..."
1,0,"Named for George Washington, the first U.S. pr..."
2,0,"The state, which is bordered on the west by th..."
3,0,Olympia is the state capital.
4,0,The state's largest city is Seattle.
5,0,Washington is often referred to as Washington ...
6,1,"Washington is the 18th largest state, with an ..."
7,1,Approximately 60 percent of Washington's resid...
8,1,The remainder of the state consists of deep te...


###Clean Raw Text
As usual, we'll need to clean our raw text before we can compute a feature vector representation for each sentence.

Run the code cell below to add our `get_clean_text()` function to your Python program.

In [33]:
#define a function that will clean the raw input text in preparation for analysis. Returns a tuple containing
#both the cleaned text and the total number of words in the cleaned text.
def get_clean_text(raw_text):
  #find any period-separated acronyms (e.g., 'U.S.A', 'L.A.', etc.)
  period_separated_acronyms = re.findall(r'(?:[A-Z]\.){2,}', raw_text)
  #remove periods from any period-separated acronyms
  for i in range(len(period_separated_acronyms)):
    acronym = period_separated_acronyms[i].replace('.', '')
    raw_text = raw_text.replace(period_separated_acronyms[i], acronym)
  #remove all numbers from the text using a regular expression
  text = re.sub(r'[0-9]', ' ', raw_text)
  #remove all underscores from the text
  text = re.sub(r'\_', ' ', text)
  #remove anything else in the text that isn't a word character or a space (e.g., punctuation, special symbols, etc.)
  text = re.sub(r'[^\w\s]', ' ', text)
  #remove any excess whitespace
  for _ in range(10):
    text = text.replace('  ', ' ')
  #remove any leading or trailing space characters
  text = text.strip()
  #split the text into a list of words
  words = text.split()
  #convert all non-acronyms to lowercase
  for i in range(len(words)): #for each index in the words collection
    word = words[i] #define the current word
    if len(word) > 1 and len(word) < 7: #if this word is two to six characters long
      if word.isupper() == False: #if at least one character in this word is not uppercase
        #this word is not an acronym because it is not all uppercase, so convert it to lowercase
        words[i] = word.lower()
    else: #this word is not an acronym because it consists of one letter or more than six letters, so convert it to lowercase
      words[i] = word.lower()
  #return the cleaned text and the number of words in the cleaned text
  return (' '.join(words), len(words))

Now that the `get_clean_text()` function has been made available to your Python program, you're ready to clean the raw text of each sentence.

Run the code cell below to clean the raw text of each sentence and save the resulting cleaned text and the total number of words in the sentence to new columns in the dataframe.

In [34]:
#clean the raw text of each sentence and save the resulting cleaned text and total number of words for
#each sentence in new dataframe columns named 'clean_text' and 'total_words'.
df[['clean_text', 'total_words']] = [get_clean_text(raw_text) for raw_text in df.raw_text]

#show the first few rows in the dataframe
df.head()

,paragraph_id,raw_text,clean_text,total_words
sentence_id,,,,
0,0,"Washington, officially the State of Washington...",washington officially the state of washington ...,19
1,0,"Named for George Washington, the first U.S. pr...",named for george washington the first US presi...,43
2,0,"The state, which is bordered on the west by th...",the state which is bordered on the west by the...,40
3,0,Olympia is the state capital.,olympia is the state capital,5
4,0,The state's largest city is Seattle.,the state s largest city is seattle,7


###Sentence Length
Research has shown that there is an inverse relationship between the number of words in a sentence and how easily a typical reader can understand the sentence. Specifically:
* Sentences containing approximately 11 words are rated as *easy to understand*.
* Sentences containing approximately 21 words are rated as *somewhat difficult to understand*.
* Sentences containing approximately 25 words are rated as *difficult to understand*.
* Sentences containing approximately 29 words or more are rated as *very difficult to understand*.

In light of this information, let's evaluate the average sentence length among the sentences in our article about the State of Washington.

**TASK 03**:
>Write a line of code in the cell below that will display the average number of words per sentence.

**QUESTION 03**:
>What is the average number of words per sentence among the sentences in the article about the State of Washington? Report your answer using three decimals of precision (e.g., 18.678).

In [44]:
#display the average number of words per sentence
for i in df:
  #print(raw_text['total_words'])
  print(df['total_words'])
  #24.636

sentence_id
0     19
1     43
2     40
3      5
4      7
5     18
6     23
7     40
8     40
9     19
10    25
11     6
12    19
13    36
14    29
15     9
16    22
17    26
18    14
19    13
20    59
21    30
Name: total_words, dtype: int64
sentence_id
0     19
1     43
2     40
3      5
4      7
5     18
6     23
7     40
8     40
9     19
10    25
11     6
12    19
13    36
14    29
15     9
16    22
17    26
18    14
19    13
20    59
21    30
Name: total_words, dtype: int64
sentence_id
0     19
1     43
2     40
3      5
4      7
5     18
6     23
7     40
8     40
9     19
10    25
11     6
12    19
13    36
14    29
15     9
16    22
17    26
18    14
19    13
20    59
21    30
Name: total_words, dtype: int64
sentence_id
0     19
1     43
2     40
3      5
4      7
5     18
6     23
7     40
8     40
9     19
10    25
11     6
12    19
13    36
14    29
15     9
16    22
17    26
18    14
19    13
20    59
21    30
Name: total_words, dtype: int64


### Compute TF-IDF Scores & Build the Vocabulary
Now we're ready to compute the TF-IDF scores for each sentence. Note that in previous assignments, we've computed TF-IDF scores for each *article* in a *corpus*. Our current task is conceptually similar, except that we'll be computing a TF-IDF vector for each *sentence* within an *article*. Each of these feature vectors, then, will describe the semantic content of its associated sentence relative to the article as a whole. This approach is necessary because extractive text summarization relies on snetence-level vectors instead of document-level vectors. 

Run the code cell below to build the vocabulary and compute and add a TF-IDF vector for each sentence to the dataframe.

In [45]:
#build the vocabulary of unique words and compute TF-IDF scores for each sentence
vectorizer = TfidfVectorizer(lowercase=False)
sentence_tfidf_scores = np.array(vectorizer.fit_transform(df.clean_text).todense())
vocabulary = vectorizer.vocabulary_

#add each sentence's vector of TF-IDF scores to the dataframe
df['tfidf_scores'] = [tfidf_scores for tfidf_scores in sentence_tfidf_scores]

#display the first few rows
df.head()

,paragraph_id,raw_text,clean_text,total_words,tfidf_scores
sentence_id,,,,,
0,0,"Washington, officially the State of Washington...",washington officially the state of washington ...,19,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,"Named for George Washington, the first U.S. pr...",named for george washington the first US presi...,43,"[0.0, 0.1316039800009417, 0.0, 0.0, 0.16478488..."
2,0,"The state, which is bordered on the west by th...",the state which is bordered on the west by the...,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1708030013569..."
3,0,Olympia is the state capital.,olympia is the state capital,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0,The state's largest city is Seattle.,the state s largest city is seattle,7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


###Compute a Centroid Representation of the Article
Next, we'll compute the centroid of the TF-IDF vectors for all of the sentences in the article. This centroid vector will be the average of all of the sentence-level TF-IDF vectors, and will hence represent the semantic content of the article as a whole. This step is necessary because we'll be using **Maximum Marginal Relevance (MMR)** to identify the most representative sentence from each passage of text, and MMR works by choosing the sentence from each passage that is most similar to the document overall, while minimizing redundancy among the set of chosen sentences.

Run the code cell below to compute a centroid vector for all of the sentences in the article.

In [47]:
#compute the centroid of the TF-IDF vectors for all of the sentences in the article
centroid = np.reshape(np.mean(df.tfidf_scores, axis=0), (1, -1))

###Calculating Maximum Marginal Relevance (MMR)
Next, let's implement a function that will identify the sentence with the Maximum Marginal Relevance (MMR) score in each paragraph. The MMR formula is used to assign each sentence in a paragraph a score based on:
1. How similar the sentence is to the article as a whole (i.e., to the article's centroid); and
2. How redundant the semantic content of each sentence is relative to the semantic content of any already-chosen summary sentences.

The idea with MMR is thus to find a group of sentences that summarize the overall article without being excessively redundant, which seems to be a reasonable strategy.

Run the code cell below to add the `get_extractive_summary()` funtion to your Python program. Note that this function has a `lambda_` parameter that controls the level of relevance vs. redundancy among the sentences in the summary. Values of this parameter can range from 0.0 to 1.0. Smaller values of this parameter yield a summary whose sentences are more similar to the article as a whole, but which may also contain more redundant information. Larger values, by contrast, will yield a summary whose sentences will be less cohesive, but which will express a wider variety of content.

In [48]:
#define a function that will generate an extractive summary of the article's text by identifying the best 
#representative, non-redundant sentence in each paragraph; i.e., the sentence with the Maximum Marginal Relevance (MMR).
def get_extractive_summary(lambda_):
  #NOTE: the lambda parameter controls the level of relevance vs. redundancy among the sentences in the summary
  summary_sentences = []
  for paragraph_id in range(len(paragraphs)): #for each paragraph ID
    #extract the TF-IDF scores for this paragraph's sentences from the dataframe
    df_sentences = df[['tfidf_scores']][df.paragraph_id == paragraph_id]
    #identify the sentence with the Maximum Marginal Relevance (MMR) for this paragraph
    maximum_marginal_relevance = -np.inf #holds the MMR for the sentences in this paragraph
    best_sentence = None #holds the sentence with the MMR
    for sentence in df_sentences.itertuples(): #for each sentence in this paragraph
      #calculate the cosine similarity between this sentence's TF-IDF scores and the article's centroid
      similarity_to_article = cosine_similarity(np.reshape(sentence.tfidf_scores, (1, -1)), centroid)[0][0]
      #calculate the maximum cosine similarity between this sentence and any already-chosen summary sentences
      max_similarity_to_summary_sentence = -np.inf
      for sentence_id, summary_sentence_tfidf_scores in summary_sentences:
        similarity_to_summary_sentence = cosine_similarity(np.reshape(sentence.tfidf_scores, (1, -1)), np.reshape(summary_sentence_tfidf_scores, (1, -1)))[0][0]
        if similarity_to_summary_sentence > max_similarity_to_summary_sentence:
          max_similarity_to_summary_sentence = similarity_to_summary_sentence
      #compute the marginal relevance for this sentence
      marginal_relevance = ((1 - lambda_) * similarity_to_article) - (lambda_ * max_similarity_to_summary_sentence)
      if marginal_relevance > maximum_marginal_relevance:
        maximum_marginal_relevance = marginal_relevance
        best_sentence = (sentence.Index, sentence.tfidf_scores)
    #add the sentence with the Maximum Marginal Relevance (MMR) for this paragraph to the collection of summary sentences
    summary_sentences.append(best_sentence)

  #construct and return the summary of the article
  article_summary = ''
  for sentence_id, _ in summary_sentences:
    article_summary += df.iloc[sentence_id]['raw_text'] + ' '
  return article_summary.strip()


###Generate Extractive Text Summaries
We're finally ready to generate extractive summaries of our article about the State of Washington. Yay!

**TASK 04**:
>Write a line of code in the cell below that will generate an extractive summary of the article about the State of Washington using a value of the `lambda_` parameter of 0.2. This value will yield a summary that emphasizes similarity between each sentence and the article as a whole.

**QUESTION 04**:
>Which topics are mentioned in the extractive summary when the value of the lambda parameter is 0.2?
* Agricultural products grown in Washington, such as apples, hops, and pears.
* Mount Rainier.
* Manufacturing industries in Washington, such as aircraft and shipbuilding.
* Wine production in Washington.
* Washington's high life expectancy and low unemployment.

In [49]:
#display a summary of the article using a value of 0.2 for the lambda parameter
get_extractive_summary(.2)

"Washington, officially the State of Washington, is a state in the Pacific Northwest region of the Western United States. Washington is the second most populous state on the West Coast and in the Western United States, after California. Washington is the nation's largest producer of apples, hops, pears, red raspberries, spearmint oil, and sweet cherries, and ranks high in the production of apricots, asparagus, dry edible peas, grapes, lentils, peppermint oil, and potatoes. Manufacturing industries in Washington include aircraft and missiles, shipbuilding, and other transportation equipment, food processing, metals and metal products, chemicals, and machinery. Washington is one of the wealthiest and most socially liberal states in the country."

**TASK 05**:
>Write a line of code in the cell below that will generate an extractive summary of the article about the State of Washington using a value of the `lambda_` parameter of 0.8. This value will yield a summary that emphasizes greater diversity among the topics that appear in the summary.

**QUESTION 05**:
>Which topics are mentioned in the extractive summary when the value of the lambda parameter is 0.8?
* Agricultural products grown in Washington, such as apples, hops, and pears.
* Mount Rainier.
* Manufacturing industries in Washington, such as aircraft and shipbuilding.
* Wine production in Washington.
* Washington's high life expectancy and low unemployment.

In [50]:
#display a summary of the article using a value of 0.8 for the lambda parameter
get_extractive_summary(.8)

"Washington, officially the State of Washington, is a state in the Pacific Northwest region of the Western United States. Mount Rainier, an active stratovolcano, is the state's highest elevation, at almost 14,411 feet, and is the most topographically prominent mountain in the contiguous U.S. Washington ranks second only to California in wine production. Manufacturing industries in Washington include aircraft and missiles, shipbuilding, and other transportation equipment, food processing, metals and metal products, chemicals, and machinery. The state consistently ranks among the best for life expectancy and low unemployment."

##Abstractive Text Summarization
We'll next turn our attention to ***abstractive text summarization***, in which our goal is to generate new sentences that are semantically similar to the original text, but which did not appear in the original text. For this lab assignment, we'll be using n-gram language models and conditional probabilities for this purpose. If each word appears in the summary text with approximately the same probability as it appeared in the original text, then the generated summary text can be said to approximate the original text.

### Load Data
For this part of the lab assignment, we'll be using the complete text of the books *The Adventures of Sherlock Holmes* and *The Memoirs of Sherlock Holmes* as the basis for generating our abstractive summaries. 

After uploading the `Sherlock Holmes.txt` file, run the code cell below to make the text of these two Sherlock Holmes books available to your Python program. This code cell will also clean up the linefeed characters in the original text, and will use NLTK's sentence tokenizer to split the text into sentences.

In [51]:
#load raw text from a file
with open('Sherlock Holmes.txt', 'r') as f:
  text = f.read()

#replace linefeed characters
text = text.replace('\n\n', ' ')
text = text.replace('\n', ' ')

#split text into a list of sentences
sentences = sent_tokenize(text)

**TASK 06**:
>Write a line of code in the cell below that will display the total number of sentences in the Sherlock Holmes books.

**QUESTION 06**:
>How many sentences are in the Sherlock Holmes books?

In [52]:
#display the total number of sentences in the Sherlock Holmes books
print(len(sentences))

9097


###Tokenize Sentences Into Lists
Next, we'll tokenize each sentence into a list whose elements contain the words and symbols that together comprise the sentence. This is necessary because NLTK's `ngrams()` function (which we'll use to generate n-grams for our language models) requires input sentences to be in the form of tokenized lists.

Run the code cell below to convert all of the sentences from the Sherlock Holmes books into lists by using NLTK's word tokenizer.

In [74]:
#tokenize each sentence into a list of words and symbols
sentences = [word_tokenize(sentence) for sentence in sentences]

TypeError: ignored

**TASK 07**:
>Write some code in the cell below that will compute and display the average number of words per sentence for the sentences in the Sherlock Holmes books.

**QUESTION 07**:
>How does the average number of words per sentence in the Sherlock Holmes books compare to the average number of words per sentence in the article about the State of Washington?
* On average, the Sherlock Holmes books have more words per sentence than the article about the State of Washington.
* On average, the Sherlock Holmes books have fewer words per sentence than the article about the State of Washington.

In [76]:
#calculate and display the average number of words per sentence in the Sherlock Holmes books
sum = 0
for i in range(len(sentences)-1):
  sum += len(sentences[i])
print(sum/9097)
#26.256 - more words in sherlock holmes

26.256238320325384


###Convert Sentences to N-Grams
Next, let's write a function that will convert a tokenized sentence into a list of n-grams of size `n`. Thus far in our course we've focused primarily on unigrams (1-grams), but these 1-grams do not encode any context because they are just single words. To add more context, we can consider short sequences of words. For example, with `n = 2`, we can capture more subtle concepts such as *bad movie* or *dark chocolate*. With `n = 3` we can capture even more subtle concepts such as *My favorite book* or *I like Python*. Using these short sequences of words, we will be able to calculate the probability of any word being the next word to appear in the sequence. For example, what word most likely comes next in this sequence?: *My favorite book ________*

Run the code cell below to define a function that will convert a tokenized sentence into a list of n-grams of size `n`. Note that the function left-pads each sentence with an appropriate number of `<s>` start-of-sentence tags, depending on the value of `n`. This will allow us to figure out the probability of any word being the first word in a sentence. We also append an `</s>` end-of-sentence tag to the end of every sentence so that we will be able to identify n-grams that appear at the end of sentences. Note also that the n-grams are stored as tuples with the format: `((n - 1 previous words), next word in sequence)`. This format will be very useful when we begin calculating the probabailities of different words appearing next in a sequence.

In [77]:
#define a function that will convert a tokenized sentence into a list of n-grams of size n.
def get_ngrams(sentence, n):
  #left-pad the sentence with an appropriate number of start-of-sentence tags. This is necessary
  #to allow the conditional probability of the first word in each sentence to be computed.
  sentence = (n - 1) * ['<s>'] + sentence
  #right-pad the sentence with an end-of-sentence tag. This is necessary to allow the conditional
  #probability of ending the sentence to be computed.
  sentence.append('</s>')
  #generate a list of n-grams for the sentence
  n_grams = list(ngrams(sentence, n))
  #convert n-grams into tuples with the format: ((n - 1 previous words), next word in sequence)
  n_grams = [((n_gram[:-1]), n_gram[-1]) for n_gram in n_grams]
  return n_grams

**TASK 08**:
>Write a line of code in the cell below that will display all of the 2-grams (i.e., `n = 3`) for the sentence at index location 999 within the `sentences` collection.

**QUESTION 08**:
>What is the first 3-gram for the sentence that appears at index location 999 in the `sentences` collection?

In [78]:
#display all of the 3-grams for the sentence that appears at index location 999 in the 'sentences' list
get_ngrams(sentences[999],3)

[(('<s>', '<s>'), 'You'),
 (('<s>', 'You'), 'have'),
 (('You', 'have'), 'really'),
 (('have', 'really'), 'done'),
 (('really', 'done'), 'very'),
 (('done', 'very'), 'well'),
 (('very', 'well'), 'indeed'),
 (('well', 'indeed'), '.'),
 (('indeed', '.'), '</s>')]

###Define Punctuation Symbols
Next, we'll define what constitutes a punctuation symbol within our text. Since we're not cleaning this text in the same way that we did when working with feature vectors, all of the punctuation symbols remain in the text. For the most part, we can use Python's standard set of punctuation symbols, but we'll need to add a few more to accommodate directional closing double-quotes and single-quotes.

Run the code cell below to define the punctuation symbols.

In [79]:
#define punctuation symbols
punctuation = string.punctuation + '”’'

###Define a Language Model Class
We've now reached a point where we can define a class that will allow us to construct and work with language models based on n-grams. Each language model is constructed based on a set of input sentences and word
sequences (n-grams) of length `n`. The language model consists of a collection of word sequences of length 
(n - 1) and the set of possible words that might immediately follow each word sequence. Each candidate word that might follow a particular sequence of words has a specific probability of being the next word in the sequence.

Run the code cell below to add the `Language_Model()` class to your Python project.

***Study the code and the comments in this class very carefully so that you will understand how the language model is built and how the conditional probabilities are calculated!***

In [80]:
#Define a language model class. A language model is constructed based on a set of input sentences and word
#sequences (n-grams) of length n. The language model consists of a collection of word sequences of length 
#(n - 1) and the set of possible words that might follow each word sequence. Each candidate word that might 
#follow a particular sequence of words has a specific probability of being the next word in the sequence.
class Language_Model():
  #define the class's initialization function
  def __init__(self, n, sentences): #n = the size of n-grams to use; sentences = the tokenized sentences from which to construct the language model.
    self.end_of_sentence_tag = '</s>' #the tag that marks the end of a sentence
    self.n = n #the number of words that constitute an n-gram
    self.n_gram_count = {} #a dictionary that holds the number of times each n-gram has been observed in the text. Keys = n-grams, values = n-gram counts
    self.sentences = sentences #the sentences from which to construct the language model
    self.start_of_sentence_sequence = tuple(['<s>' for _ in range(n - 1)]) #a tuple containing the appropriate number of successive start-of-sentence tags (based on n) to indicate the beginning of a new sentence
    self.word_sequences = {} #a dictionary containing sequences of words (keys) and a list of candidate words that may follow each sequence (values)
    
    #build the language model
    for sentence in sentences: #for each sentence
      #get the n-grams for this sentence
      n_grams = get_ngrams(sentence, n)
      #for each n-gram in this sentence
      for n_gram in n_grams:
        #if this n-gram has been seen previously
        if n_gram in self.n_gram_count:
          self.n_gram_count[n_gram] += 1 #increment the number of times this n-gram has been observed
        else: #if this is the first time this n-gram has been seen
          self.n_gram_count[n_gram] = 1 #add this n-gram to the dictionary and set its number of observations to 1
        #extract the sequence of previous words and the next word from this n-gram
        word_sequence, next_word = n_gram
        #if this sequence of words has already been observed
        if word_sequence in self.word_sequences:
          self.word_sequences[word_sequence].append(next_word) #add the next word to this sequence's list of next words
        else: #if this sequence of words has not yet been observed
          self.word_sequences[word_sequence] = [next_word] #add this word sequence to the dictionary and initialize its list of next words
    
    #calculate conditional probabilities for all of the words that might follow each sequence of words
    for word_sequence in self.word_sequences:
      #get the raw word frequencies for each word that might follow this word sequence
      freq_dist = FreqDist(self.word_sequences[word_sequence])
      #compute the conditional probabilities for each word and store them in a list that is sorted
      #from the most probable word to the least probable word
      word_probabilities = [(word, freq_dist.freq(word)) for word, frequency in freq_dist.most_common()]
      #replace the previous list of words that might follow this word sequence with the list of words and probabilities
      self.word_sequences[word_sequence] = word_probabilities

  #define a function that will return the next word (or symbol) in a sequence. The likelihood of any of the
  #possible words being returned depends on its conditional probability, given the word sequence.
  def get_next_word(self, word_sequence):
    #get a random number between 0 and 1 to serve as the probability threshold
    random_probability_threshold = np.random.random()
    #define a variable to hold the cumulative probability
    cumulative_probability = 0.0
    #define a variable to hold the next word in the sequence
    next_word = None
    #determine which word should appear next in the sequence based on the words' conditional probabilities,
    #for each possible next word
    for word, word_probability in self.word_sequences[word_sequence]:
      #add this word's probability of being next in the sequence to the cunulative probability
      cumulative_probability += word_probability
      #if the cumulative probability exceeds the randomly chosen probability threshold
      if cumulative_probability >= random_probability_threshold:
        #assign the current word to be the next word in the sequence, and exit the loop immediately
        next_word = word
        break
    return next_word
  
  #define a function that will generate a sentence based on the language model
  def generate_sentence(self):
    #define a variable to hold the words (and symbols) that comprise the sentence
    sentence = ''
    #initialize the word sequence to the start-of-sentence sequence
    word_sequence = self.start_of_sentence_sequence
    #get the first word in the sentence
    word = self.get_next_word(word_sequence)
    #while the end of the sentence has not yet been reached
    while word != self.end_of_sentence_tag:
      #add this word (or symbol) to the sentence
      sentence += ' ' + word
      #construct the next word sequence
      word_sequence = word_sequence[1:] + (word,)      
      #get the next word (or symbol) in the sentence
      word = self.get_next_word(word_sequence)
    #cleanup punctuation and spacing
    for symbol in punctuation:
      sentence = sentence.replace(' ' + symbol, symbol)
    for symbol in '“‘':
      sentence = sentence.replace(symbol + ' ', symbol)
    sentence = sentence.replace('n’ t', 'n’t')
    sentence = sentence.replace('’ s ', '’s ')
    sentence = sentence.replace('’ m ', '’m ')
    return sentence.strip()


###Generate Language Models
We now have everything that we need to generate language models based on n-grams. Yay!

Run the code cell below to generate a language model using 3-grams (i.e., `n = 3`).

In [88]:
#generate a language model using 3-grams
n3_model = Language_Model(3, sentences)


**TASK 09**:
>Write a line of code in the cell below that will display the words that are most likely to appear as the first word in a sentence in the Sherlock Holmes books. ***Tip***: You will need to get the values out of the `n3_model` language model's `word_sequences[]` collection for the language model's `start_of_sentence_sequence` property.

**QUESTION 09**:
>Which <u>word</u> (as opposed to a punctuation symbol) is most likely to appear as the first word in a sentence in the Sherlock Holmes books?
* It
* The
* I
* You

In [111]:
#display the words that are most likely to appear as the first word in a sentence
x = n3_model.start_of_sentence_sequence
t=1
it_count = 0
the_count = 0
i_count = 0
you_count = 0
while t < 1000:
  if (n3_model.get_next_word(x)) == 'It':
    it_count += 1
  if (n3_model.get_next_word(x)) == 'The':
    the_count += 1
  if (n3_model.get_next_word(x)) == 'I':
    i_count += 1
  if (n3_model.get_next_word(x)) == 'You':
    you_count += 1
  t+=1
print(it_count)
print(the_count)
print(i_count)
print(you_count)
#

47
71
94
23


###Generate Abstractive Sentences Using the Language Model
Now it's time to use our language model to generate some new sentences that don't appear in the original text. Nevertheless, our new sentences should seem  similar to the kinds of sentences that we would expect to appear in a Sherlock Holmes book, given that the model was trained using Sherlock Holmes books.

Run the code cell below to generate five new sentences using our `n = 3` language model. Since the value of `n` is relatively small in this particular language model, we should not expect the sentences to make perfect sense.

In [18]:
#set the seed for the random number generator (to ensure consistent results)
np.random.seed(123)

#generate five sentences for the n=3 language model
for i in range(5):
  sentence = n3_model.generate_sentence()
  print(sentence, '\n')

NameError: ignored

One of the interesting things about language models based on n-grams is that as the value of `n` increases, the newly generated sentences seem more and more natural. The problem, however, is that as the value of `n` increases, so too does the probability of exactly reproducing one of the actual sentences from the original text. For this reason, it is generally recommended to use values of `n` of between 3 and 5.

**TASK 10**:
>Write some code in the cell below that will train a language model for `n = 5`, and then generate five new sentences using your `n = 5` language model. ***NOTE***: Do <u>not</u> change the random seed value that appears in the cell. If you do, you will be unable to answer Question 10.

**QUESTION 10**:
>What is the fifth sentence generated by the `n = 5` model?

In [ ]:
#generate a language model using 5-grams


In [ ]:
#set the seed for the random number generator (to ensure consistent results)
np.random.seed(123)

#generate five sentences for the n=5 model


As you can see, the sentences generated by the `n = 5` model seem much more natural and understandable than the sentences generated by the `n = 3` model. Nevertheless, it is important to remember that these sentences are being generated statistically -- our Python program does not actually ***understand*** the meaning of the sentences that it is writing!

##End of Lab Assignment 03!